#### Model: $dX_t = \theta_{1}(\theta_{2}-X_{t})dt + \sigma dW_{t}$, $Y_{t}|X_{t} \sim \mathcal{N}(X_{t},\theta_{2}^2)$

In [55]:
import numpy as np
from numpy import math

####### Model Specification ########
theta = np.array([0,0,0.01])
sigma = 2
initial_val = 5
####################################

def diff_coef(x, dt, dw):
    return sigma*np.sqrt(dt)*dw

def drift_coef(x, dt):
    return theta[0]*(theta[1]-x)*dt

# Log-scaled unnormalized likelihood function p(y|x)
def likelihood_logscale(y, x):
    d = (y-x)
    gn = -1/2*(d**2/theta[2]**2)
    return gn

# Resampling - input two-dimensional particle x
def resampling(weight, gn, N, x):
    ess = 1/((weight**2).sum())
    if ess <= (N/2):
        ## Sample with uniform dice
        dice = np.random.random_sample(N)
        ## np.cumsum obtains CDF out of PMF
        bins = np.cumsum(weight)
        ## np.digitize gets the indice of the bins where the dice belongs to 
        x_hat = x[:,np.digitize(dice,bins)]
        ## after resampling we reset the accumulating weight
        gn = np.zeros(N)
    if ess > (N/2):
        x_hat = x
    
    return x_hat, gn

# Sampling out according to the weight
def sample_output(weight, N, x):
    ## Sample with uniform dice
    dice = np.random.random_sample(1)
    ## np.cumsum obtains CDF out of PMF
    bins = np.cumsum(weight)
    ## np.digitize gets the indice of the bins where the dice belongs to 
    x_hat = x[:,np.digitize(dice,bins)].reshape(x.shape[0])
    ## return the sampled particle path
    return x_hat
    

def get_discrete_ind(t,l):
    return int(t*2**l)

def initial_path_gen(l,T):
    hl = 2**(-l)
    time_len = get_discrete_ind(T,l)
    ## Initial value
    un1 = np.zeros(time_len+1) + initial_val
    for t in range(T):
        ind = get_discrete_ind(t,l)
        for dt in range(2**l):
            dw = np.random.randn(1)
            un1[ind+dt+1] = un1[ind+dt] + drift_coef(un1[ind+dt], hl) + diff_coef(un1[ind+dt], hl, dw[0])
    return un1

def sig_mean(t):
    return initial_val*np.exp(-theta[0]*t) + theta[1]*(1-np.exp(-theta[0]*t))

## Used only when theta[0] != 0
def sig_var(t):
    return (sigma**2 / (2*theta[0])) * (1-np.exp(-2*theta[0]*t))

def coef(x, y): 
    # number of observations/points 
    n = np.size(x) 
  
    # mean of x and y vector 
    m_x, m_y = np.mean(x), np.mean(y) 
  
    # calculating cross-deviation and deviation about x 
    SS_xy = np.sum(y*x) - n*m_y*m_x 
    SS_xx = np.sum(x*x) - n*m_x*m_x 
  
    # calculating regression coefficients 
    b_1 = SS_xy / SS_xx 
    b_0 = m_y - b_1*m_x 
  
    return(b_0, b_1) 


def gen_data(T):
    Y = np.zeros(T+1)
    for t in range(T+1):
        std = np.sqrt(sig_var(t) + theta[2]**2)
        Y[t] = sig_mean(t) + std * np.random.randn(1)
    return Y

def grad_exact(theta,T,Y):
    m_x = np.zeros(T+1)
    v_x = np.zeros(T+1)
    grad_m = np.zeros((T+1,3))
    grad_v = np.zeros((T+1,3))
    grad_vt3 = np.zeros((T+1,3))

    grad1 = np.zeros((T+1,3))
    grad2 = np.zeros((T+1,3))
    for t in range(1,T+1):
        m_x[t] = initial_val*np.exp(-theta[0]*t) + theta[1]*(1-np.exp(-theta[0]*t))        
        v_x[t] = (sigma**2 / (2*theta[0])) * (1-np.exp(-2*theta[0]*t))

        grad_m[t,0] = (theta[1]-initial_val)*t*np.exp(-theta[0]*t)
        grad_m[t,1] = 1-np.exp(-theta[0]*t)
        grad_v[t,0] = -((sigma**2)/(2*theta[0]**2))*(1-np.exp(-2*theta[0]*t)) + (t*(sigma**2)/theta[0])*np.exp(-2*theta[0]*t)
        grad_vt3[t] = grad_v[t] + np.array([0,0,theta[2]*2])

        grad1[t] = -0.5*grad_vt3[t]/(v_x[t]+theta[2]**2)    
        grad2[t] = (-0.5)*( 2*(Y[t]-m_x[t])*(-grad_m[t])*(v_x[t]+theta[2]**2) -  grad_vt3[t]*(Y[t]-m_x[t])**2 ) / (v_x[t]+theta[2]**2)**2
        
    return np.sum(grad1+grad2,axis=0) 

#### Cond-PF function

In [56]:
def Conditional_particle_filter(l,T,N,particle_path):
    
    hl = 2**(-l)
    time_len = get_discrete_ind(T,l)
    un = np.zeros((time_len+1,N)) + initial_val
    un_hat = np.zeros(un.shape) + initial_val
    gn = np.zeros(N)
    for t in range(T):
        ind = get_discrete_ind(t,l)
        un[:ind+1] = un_hat[:ind+1]
        
        for dt in range(2**l):
            dw = np.random.randn(N)
            un[ind+dt+1] = un[ind+dt] + drift_coef(un[ind+dt], hl) + diff_coef(un[ind+dt], hl, dw)
            
        # Main point for conditional PF is that the last particle is fixed, and it joins the resampling process
        next_ind = get_discrete_ind(t+1,l)
        un[:,-1] = particle_path
        
        # Cumulating weight function
        gn = likelihood_logscale(Y[t+1], un[next_ind]) + gn
        what = np.exp(gn-np.max(gn))
        wn = what/np.sum(what)
        
        # Wasserstein resampling
        un_hat[:next_ind+1], gn = resampling(wn, gn, N, un[:next_ind+1])
        un_hat[:,-1] = particle_path
        
    # Sample out a path and output it
    return sample_output(wn, N, un)

#### RTS function, usful when $\theta_{2}=0$

In [57]:
def RTS_OU(theta,T,Y):
    ## Paramter calculation
    F = np.exp(-theta[0])
    Q = ((sigma**2)/(2*theta[0])) * (1-np.exp(-2*theta[0]))
    H = 1
    R = theta[2]**2
    
    mu = np.zeros(T+1)
    V = np.zeros(T+1)
    P = np.zeros(T+1)
    K = np.zeros(T+1)
    ## Initialize: At time 0 no data is available thus X_0|Y_0 \sim X_0
    mu[0]=initial_val
    V[0]=0
    ## P[0] is variance of X_1
    P[0] = F*V[0]*F + Q
    ## Loop 
    for t in range(1,T+1): 
        K[t] = P[t-1]*H / (H*P[t-1]*H + R)
        mu[t] = F*mu[t-1] + K[t]*(Y[t]-H*F*mu[t-1])
        V[t] = (1-K[t]*H)*P[t-1]
        P[t] = F*V[t]*F + Q

    ## RTS loop
    C = np.zeros(T+1)
    Vhat = np.zeros(T+1)
    muhat = np.zeros(T+1)
    muhat[-1] = mu[-1]
    Vhat[-1] = V[-1]
    for t in reversed(range(T)):
        C[t] = V[t]*F / P[t]
        muhat[t] = mu[t] + C[t]*(muhat[t+1]-F*mu[t])
        Vhat[t] = V[t] + C[t]*(Vhat[t+1]-P[t])*C[t]
    return muhat,Vhat

#### $G_{\theta}(x) = \nabla_{\theta}\log \frac{d\mathbb{P}_{\theta}}{d\mathbb{Q}} + \sum_{t=1}^{T}\nabla_{\theta}\log g_{\theta}(y_{t}|x_{t})$
#### $G_{\theta}(x)_{3} = \sum_{t=1}^{T}\frac{d}{d\theta_{3}}\log g_{\theta}(y_{t}|x_{t})$

In [58]:
def grad_lgT(theta,x_seq): 
    T = int(x_seq.shape[0]-1)
    grad_val = np.zeros(T+1)
    for t in range(1,T+1):
        grad_val[t] =  -1/theta[2] + (Y[t]-x_seq[t])**2/(theta[2]**3)
    return np.sum(grad_val[1:])

def Gfunc_discrete(l,x_discrete):
    
    gtheta = np.zeros(3)
    ## First & Second entry
    hl = 2**(-l)    
    gtheta[0] += sigma**(-2) * np.sum((theta[1] - x_discrete[:-1]) * (x_discrete[1:]-x_discrete[:-1]))
    gtheta[0] -= sigma**(-2) * theta[0] * np.sum((theta[1]-x_discrete[:-1])**2) * hl
    
    gtheta[1] += sigma**(-2)*theta[0]*(x_discrete[-1]-x_discrete[0])
    gtheta[1] -= sigma**(-2) * theta[0]**2 * np.sum(theta[1]-x_discrete[:-1]) * hl
        
    ## Third entry
    x_seq = x_discrete[0::2**l]
    T = x_seq.shape[0]-1
    gtheta[2] = grad_lgT(theta,x_seq)
        
    return gtheta

In [59]:
def Gl(l,x_discrete):
    
    gtheta = np.zeros(3)
    ## First & Second entry
    hl = 2**(-l)    
    ## Get unit path & Time
    x_seq = x_discrete[0::2**l]
    T = x_seq.shape[0]-1

    ## \int x_t dt approximation
    inte_x = np.sum(x_discrete[:-1]*hl)
    ## \int x_t^2 dt approximation
    inte_sx = np.sum(x_discrete[:-1]**2 * hl)
    ## difference
    diff_x = x_discrete[-1] - x_discrete[0]
    ## squared difference
    sdiff_x = x_discrete[-1]**2 -x_discrete[0]**2

    ## Entry computation
    gtheta[0] = sigma**(-2) * ( theta[1]*diff_x + 0.5*sigma**2*T - 0.5*sdiff_x - theta[0]*theta[1]**2*T + 2*theta[0]*theta[1]*inte_x - theta[0]*inte_sx )
    
    gtheta[1] = sigma**(-2) * ( theta[0]*diff_x - theta[0]**2*theta[1]*T + theta[0]**2*inte_x )
    
    gtheta[2] = grad_lgT(theta,x_seq)
    
    return gtheta

#### 1.1. Model with $\theta_{3}=1$, $\theta_{2}=0$ (so RTS can be applied)

In [128]:
####### Model Specification ########
theta = np.array([0.5,0,1])
sigma = 0.2
initial_val = 1
####################################

T = 10
Y = gen_data(T)

In [181]:
grad_exact(theta,T,Y)

array([ 0.00062469,  0.00127066, -0.09076468])

#### 1.1.1. Monte Carlo approximation of $\pi_{\theta} (G_{\theta})_{3}$: This is only useful when $\theta_{2}=0$

In [130]:
mu,V = RTS_OU(theta,T,Y)

num_mc = 100000
x_col = np.zeros((num_mc,T+1))
rep_val = np.zeros(num_mc)
pr = progressbar.ProgressBar(max_value=num_mc).start()
for i in range(num_mc):
    ## Generate a path according to marginal smoothing distribution by RTS smoother
    x_seq = np.zeros(T+1)
    for t in range(1,T+1):
        x_seq[t] = mu[t] + np.random.randn(1)*np.sqrt(V[t])
    ## Repord value of G evaluated with this realization    
    rep_val[i] = grad_lgT(theta,x_seq)
    pr.update(i+1)
pr.finish()

print('G(X_T) approximatly around',np.mean(rep_val),'+-',np.sqrt(np.var(rep_val)/num_mc)*2)

100% (100000 of 100000) |################| Elapsed Time: 0:00:11 Time:  0:00:11


G(X_T) approximatly around -4.386021259852326 +- 0.005585222780913729


In [178]:
sig_mean(0)

1.0

In [115]:
Full_Particle_filter(l,T,N)

0.11607122070033346

In [122]:
V

array([0.        , 0.0243324 , 0.03262214, 0.03544635, 0.03640852,
       0.03673632, 0.036848  , 0.03688605, 0.03689901, 0.03690343,
       0.03690494, 0.03690549, 0.03690575, 0.03690608, 0.03690689,
       0.0369092 , 0.03691597, 0.03693583, 0.03699413, 0.03716526,
       0.03766756])

In [121]:
np.var(path_col,axis=0)

array([0.        , 0.02563071, 0.03326259, 0.03612275, 0.03769614,
       0.03854013, 0.03858819, 0.03920326, 0.03846563, 0.03816246,
       0.03722968, 0.0370558 , 0.03745014, 0.03705456, 0.03704914,
       0.03766984, 0.03711735, 0.03791601, 0.03791878, 0.03828559,
       0.04004685])

#### 1.1.2. Markov Chain Monte Carlo approximation of $\pi_{\theta}^{l}(G_{\theta})_{3}$ with Cond-PF

In [117]:
l=4
initial_path = initial_path_gen(l,T)
Rep_num = 10000

path_iter = np.zeros(initial_path.shape[0])
path_iter = Conditional_particle_filter(l,T,N,initial_path)
path_col = np.zeros((Rep_num,T+1))
rep_val = np.zeros(Rep_num)
unit_path = np.zeros(T+1)


pr = progressbar.ProgressBar(max_value=Rep_num).start()
for i in range(Rep_num):
    unit_path = path_iter[0::2**l]
    rep_val[i] = grad_lgT(theta,unit_path)
    path_col[i] = unit_path
    path_iter = Conditional_particle_filter(l,T,N,path_iter)
    pr.update(i+1)
pr.finish()

print('G(X1) approximatly around',np.mean(rep_val),'+-',np.sqrt(np.var(rep_val)/Rep_num)*2)

100% (10000 of 10000) |##################| Elapsed Time: 0:02:58 Time:  0:02:58


G(X1) approximatly around 11.316612950475859 +- 0.058589823258968846


In [131]:
l=4
initial_path = initial_path_gen(l,T)
Rep_num = 1000

path_iter = np.zeros(initial_path.shape[0])
path_iter = Conditional_particle_filter(l,T,N,initial_path)
rep_val = np.zeros((Rep_num,3))
unit_path = np.zeros(T+1)


pr = progressbar.ProgressBar(max_value=Rep_num).start()
for i in range(Rep_num):
    rep_val[i] = Gfunc_discrete(l,path_iter)
    path_iter = Conditional_particle_filter(l,T,N,path_iter)
    pr.update(i+1)
pr.finish()

print('G(X) approximatly around',np.mean(rep_val,axis=0),'+-',np.sqrt(np.var(rep_val,axis=0)/Rep_num)*2)

100% (1000 of 1000) |####################| Elapsed Time: 0:00:08 Time:  0:00:08


G(X) approximatly around [ 1.50515474  0.10516891 -4.42285199] +- [0.32401847 0.48029199 0.05917162]


#### True value $\pi_{\theta}(G_{\theta}) = \nabla \log p_{\theta}(y)$

In [133]:
grad_exact(theta,T,Y)

array([ 1.42887422,  0.34248389, -4.33960349])

#### 1.2. General Model with relatively big $\theta_{3}$ values

In [142]:
####### Model Specification ########
theta = np.array([0.5,0.4,1])
sigma = 0.2
initial_val = 1
####################################

T = 10
Y = gen_data(T)

#### True value $\pi_{\theta}(G_{\theta}) = \nabla \log p_{\theta}(y)$

In [143]:
grad_exact(theta,T,Y)

array([-0.6421106 ,  2.86502946, -5.03298652])

#### MCMC approximation of $\pi_{\theta}^{l}(G_{\theta})$ Cond-PF

In [144]:
l=5
initial_path = initial_path_gen(l,T)
Rep_num = 1000

path_iter = np.zeros(initial_path.shape[0])
path_iter = Conditional_particle_filter(l,T,N,initial_path)
rep_val = np.zeros((Rep_num,3))
unit_path = np.zeros(T+1)


pr = progressbar.ProgressBar(max_value=Rep_num).start()
for i in range(Rep_num):
    rep_val[i] = Gfunc_discrete(l,path_iter)
    path_iter = Conditional_particle_filter(l,T,N,path_iter)
    pr.update(i+1)
pr.finish()

print('G(X) approximatly around',np.mean(rep_val,axis=0),'+-',np.sqrt(np.var(rep_val,axis=0)/Rep_num)*2)

100% (1000 of 1000) |####################| Elapsed Time: 0:00:15 Time:  0:00:15


G(X) approximatly around [-0.62691618  2.44093128 -5.14555165] +- [0.29107692 0.47551471 0.06405028]


In [138]:
np.var(rep_val,axis=0)

array([18.53326968, 54.51237546,  1.25748629])

#### In order to keep the variance low, we need to choose the parameters carefully

#### For $G_{\theta}(X_{t}^i)_{3}$, preferred choice is small $\sigma$ and big $\theta_{1}$ and (relatively) big $\theta_{2}$

In [225]:
####### Model Specification ########
theta = np.array([3,7,11])
sigma = 1
initial_val = 1
####################################

T = 1
Y = gen_data(T)

In [226]:
N = 1000

In [227]:
import progressbar
l=8
initial_path = initial_path_gen(l,T)
Rep_num = 1000

path_iter = np.zeros(initial_path.shape[0])
path_iter = Conditional_particle_filter(l,T,N,initial_path)
rep_val = np.zeros((Rep_num,3))
unit_path = np.zeros(T+1)
end_sample = np.zeros(Rep_num)


pr = progressbar.ProgressBar(max_value=Rep_num).start()
for i in range(Rep_num):
    rep_val[i] = Gl(l,path_iter)
    end_sample[i] = path_iter[-1]
    path_iter = Conditional_particle_filter(l,T,N,path_iter)
    pr.update(i+1)
pr.finish()

print('G(X)_1 approximatly around',np.mean(rep_val,axis=0),'+-',np.sqrt(np.var(rep_val,axis=0)/Rep_num)*2)

100% (1000 of 1000) |####################| Elapsed Time: 0:00:11 Time:  0:00:11


G(X)_1 approximatly around [-0.06943125  0.07853769 -0.03581512] +- [0.15403531 0.18666798 0.00032774]


In [228]:
grad_exact(theta,T,Y)

array([ 0.02119043,  0.06712179, -0.0358963 ])

In [229]:
## sigma=1
np.var(rep_val,axis=0)

array([5.93171942e+00, 8.71123402e+00, 2.68539728e-05])

#### Check convergence wrt to $l$ of $\pi_{\theta}^{l}(G_{\theta}^{l}) - \pi_{\theta}(G_{\theta})$

In [236]:
####### Model Specification ########
theta = np.array([0.5,0.4,1])
sigma = 0.2
initial_val = 1
####################################


T = 50
Y = gen_data(T)

In [237]:
import progressbar
l=2
initial_path = initial_path_gen(l,T)
Rep_num = 10000

path_iter = np.zeros(initial_path.shape[0])
path_iter = Conditional_particle_filter(l,T,N,initial_path)
rep_val = np.zeros((Rep_num,3))
unit_path = np.zeros(T+1)
end_sample = np.zeros(Rep_num)


pr = progressbar.ProgressBar(max_value=Rep_num).start()
for i in range(Rep_num):
    rep_val[i] = Gl(l,path_iter)
    end_sample[i] = path_iter[-1]
    path_iter = Conditional_particle_filter(l,T,N,path_iter)
    pr.update(i+1)
pr.finish()

print('G(X)_1 approximatly around',np.mean(rep_val,axis=0),'+-',np.sqrt(np.var(rep_val,axis=0)/Rep_num)*2)

100% (10000 of 10000) |##################| Elapsed Time: 0:02:31 Time:  0:02:31


G(X)_1 approximatly around [-1.5643084   9.288106    3.77657222] +- [0.15233256 0.33079512 0.05283027]


In [235]:
np.var(rep_val,axis=0)

array([2.00557997e+01, 4.50104694e+02, 4.42985515e-03])

In [233]:
grad_exact(theta,T,Y)

array([5.85715707e-04, 4.07427895e-01, 1.19812108e+00])

In [73]:
import autograd.numpy as np
from autograd import grad

def grad_auto1(theta):
    ans = 0
    for t in range(1,T+1):        
        x_mean = initial_val*np.exp(-theta[0]*t) + theta[1]*(1-np.exp(-theta[0]*t))
        x_var = (sigma**2 / (2*theta[0])) * (1-np.exp(-2*theta[0]*t))
     
        ans = ans - 0.5*np.log(theta[2]**2 + x_var) - ((Y[t]-x_mean)**2)/(2*(theta[2]**2+x_var))
    return ans

grad_auto = grad(grad_auto1)